# Public Opinion on Vaccines during COVID19 pandemic through Twitter 
Himani Anil Deshpande

Downloading Tweepy 

In [ ]:
!pip3 install tweepy --upgrade

In [ ]:
!pip install empath

In [ ]:
!pip install pyLDAvis

Importing Libraries

In [ ]:
import tweepy
import pandas as pd
import json
import string
from pprint import pprint
import requests
import re
from empath import Empath
import gensim
from gensim import corpora
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score, precision_recall_fscore_support
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


from  gensim.models import LdaMulticore


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords


Creating Client for Twitter API v2 which return request.Response as output and keeping the rate limit on


In [ ]:
client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAACzebwEAAAAACgcLsF9soPXVU15hOy4bvyY6dmo%3Dqj7BZubvoH7GEZnTomKEwCkXjq51Ah4k7Y96Vc2b7t5VoweDOE', wait_on_rate_limit=True,return_type= requests.Response)


Setting query parameters 

In [ ]:
tweet_fields = ["geo", "created_at", "id", "text", "author_id", "public_metrics", "source"]
expansions = ["geo.place_id"]
place_fields = ["country", "full_name", "contained_within"]



Building a query which is searched

In [ ]:

query = """((#CovidVaccine) OR vaccination OR vaccinations OR vaccine OR vaccines OR immunization OR vaccinate OR vaccinated OR  (#COVID-19) OR coronavirus
OR CORONA OR (covid-19 Vaccines) OR SARS-CoV-2 OR (vaccination Refusal) OR (#Pandemic) OR (vaccination drive) OR immunity OR immune OR rollout 
OR (side effect) OR immunise OR immunize
OR immunisation OR moderna OR pfizer OR biontech OR
astrazeneca OR  novavax OR #pfizerbiontech) place_country:US has:geo -is:retweet lang:en"""

# https://arxiv.org/pdf/2104.01131.pdf

# (#CovidVaccine) OR vaccination OR vaccinations OR vaccine OR vaccines OR immunization OR vaccinate OR vaccinated OR  (#COVID-19) OR coronavirus
# OR CORONA OR covid-19 OR Vaccines OR SARS-CoV-2 OR (vaccination Refusal) OR (#Pandemic) OR (vaccination drive) OR immunity OR immune OR rollout 
# OR (side effect) OR immunise OR immunize
# OR immunisation


Taking Data from all the months from September
2020 to May 2021 


In [ ]:
Timeline = {"Before_Sep": ("2020-09-30", "2020-09-01"),
            "Before_Oct":("2020-10-31", "2020-10-01") ,"Before_Nov":("2020-11-30", "2020-11-01")
,"During_Dec":("2020-12-31", "2020-12-01"),"During_Jan":("2021-01-31", "2021-01-01"),
"During_Feb":("2021-02-28", "2021-02-01"),"After_Mar":("2021-03-31", "2021-03-01") 
,"After_Apr":("2021-04-30", "2021-04-01"),"After_May":("2021-05-31", "2021-05-01")}
# (EndTime, StartTime) 


Function to fetch Archived Tweets

In [ ]:

def get_tweets(endTime , startTime,token= None ):
    return client.search_all_tweets(query = query,  end_time=endTime +"T00:00:00Z", start_time=startTime +"T00:00:00Z",
                         expansions=expansions, max_results=500,  place_fields=place_fields, 
                                  sort_order= ['relevancy'], next_token= token,
                           tweet_fields=tweet_fields)

Saving the Raw data in a Json file

In [ ]:
def save_raw_data(file_name, data):
    json_object = json.dumps(data, indent = 4)

    with open(file_name, "w") as f:
        f.write(json_object)

Converting the JSON output in a Data Frame

In [ ]:
def convert_json_to_df(tweets_json_data):
    
    data_tweet_df = pd.json_normalize(tweets_json_data['data'])#, record_path =['students'])
    geo_df = pd.json_normalize(tweets_json_data['includes']['places'])#, record_path =['students']))
    geo_df.rename(columns= {'id':'geo.place_id'}, inplace = True)
    combined_data_df = pd.merge(data_tweet_df,geo_df, on = 'geo.place_id')
    return combined_data_df

## Fetching Data from Twitter 

In [ ]:
records_cnt = 0
token = None
full_combined_df = None
for k,v in Timeline.items():
    endTime, startTime = v
#   Getting Tweets 
    tweets_json = get_tweets( endTime=endTime , startTime= startTime, token = token).json()
#     Saving the Raw JSON data
    save_raw_data("raw-data-page-"+k+".json", tweets_json)
#     Getting a count of records to verify Later with the shape of full dataframe
    records_cnt += tweets_json['meta']['result_count']
#     token= tweets_json['meta']['next_token']
    
#     Converting JSON data into Dataframe and Saving them in CSV format
    combined_data_df = convert_json_to_df(tweets_json)
    combined_data_df['Timeline'] = k
    combined_data_df.to_csv("raw_dataFrame_page_"+k+".csv")
#     Combining two Pages of Tweets's Data frames into one
    full_combined_df = pd.concat([full_combined_df,combined_data_df])

In [ ]:
full_combined_df = full_combined_df.reset_index()

In [ ]:
full_combined_df.shape

(4430, 16)

In [ ]:
records_cnt

4430

In [ ]:
full_combined_df.head()

,index,created_at,id,author_id,source,text,geo.place_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,geo.coordinates.type,geo.coordinates.coordinates,country,full_name,Timeline
0,0,2020-09-06T15:01:05.000Z,1302622857059610626,705891420134121472,Twitter for iPhone,I’m all for vaccines but this COVID one seems ...,8173485c72e78ca5,8344,60,27251,352,NaN,NaN,United States,"Atlanta, GA",Before_Sep
1,1,2020-09-28T19:55:41.000Z,1310669529723203586,77104158,Twitter for iPhone,Y’all taking that mark of the beast vaccine wh...,8173485c72e78ca5,122,22,677,14,NaN,NaN,United States,"Atlanta, GA",Before_Sep
2,2,2020-09-17T04:18:41.000Z,1306447460575911936,1057867352,Twitter for Mac,2. Ventilation: \nA good supply of outside air...,8173485c72e78ca5,0,1,0,0,NaN,NaN,United States,"Atlanta, GA",Before_Sep
3,3,2020-09-24T01:20:07.000Z,1308939234200387584,1383272101,Twitter for iPhone,BREAKING: Trump announced today he will overru...,8e9665cec9370f0f,7185,1285,23549,1049,NaN,NaN,United States,"Minneapolis, MN",Before_Sep
4,4,2020-09-13T18:46:27.000Z,1305216289552707584,789729026,Twitter for Android,They doing free Corona testing at George Floyd...,8e9665cec9370f0f,509,13,943,1,NaN,NaN,United States,"Minneapolis, MN",Before_Sep


Saving the Full Data we extracted from Twitter in a CSV file

In [ ]:
full_combined_df.to_csv("All_Original_data.csv")

Creating an extract of Columns that i will be working on


In [ ]:
full_combined_df.columns


Index(['index', 'created_at', 'id', 'author_id', 'source', 'text',
       'geo.place_id', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'geo.coordinates.type',
       'geo.coordinates.coordinates', 'country', 'full_name', 'Timeline'],
      dtype='object')

In [ ]:
full_combined_df = full_combined_df[['created_at', 'text', 'source',
       'public_metrics.retweet_count', 'public_metrics.reply_count',
       'public_metrics.like_count', 'public_metrics.quote_count',
        'country', 'full_name', 'Timeline']]

In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline
0,2020-09-06T15:01:05.000Z,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep
1,2020-09-28T19:55:41.000Z,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep
2,2020-09-17T04:18:41.000Z,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep
3,2020-09-24T01:20:07.000Z,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep
4,2020-09-13T18:46:27.000Z,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep


## Data Cleaning

Removing URL's and Usernames 

In [ ]:


def clean_usernames_links(tweet):
    
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub('@[^\s]+','',tweet)
    return tweet



<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \s
<input>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-239-eab75717a999>:5: DeprecationWarning: invalid escape sequence \s
  tweet = re.sub('http[^\s]+','',tweet)
<ipython-input-239-eab75717a999>:6: DeprecationWarning: invalid escape sequence \s
  tweet = re.sub('@[^\s]+','',tweet)


In [ ]:
full_combined_df['cleaned_tweet_text'] = full_combined_df['text'].apply(clean_usernames_links)


Removing Punctuations and keeping text in lowercase

In [ ]:

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punctuations = '['+ string.punctuation+ ']'

In [ ]:
punctuations

'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [ ]:

full_combined_df['cleaned_tweet_text'] = full_combined_df['cleaned_tweet_text'].map(lambda x: re.sub(punctuations, '', x).lower())


 Removing Stopwords 

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
full_combined_df['cleaned_tweet_text']= full_combined_df['cleaned_tweet_text'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stopwords)]))


Remove Emoticons

In [ ]:
full_combined_df["cleaned_tweet_text"] = full_combined_df["cleaned_tweet_text"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


Removing numbers and words with length less than 4 and tweet Search keywords

In [ ]:
tweet_search_keyword = ['CovidVaccine','vaccination','vaccinations','vaccine','vaccines','immunization',
'vaccinate','vaccinated','covid','COVID-19','covid-19','Vaccines','SARS-CoV-2','vaccination Refusal','Pandemic',
'vaccination drive','immunity','immune','rollout ','side effect','immunise','immunize','immunisation','moderna',
'pfizer','biontech','astrazeneca','novavax','pfizerbiontech','coronavirus','CORONA']

In [ ]:
tweet_search_keyword = [x.lower() for x in tweet_search_keyword]

In [ ]:

full_combined_df['cleaned_tweet_text'] = full_combined_df['cleaned_tweet_text'].apply(lambda x : ' '.join(i for i in x.split() if  not i.isdigit() and len(i)>4 ))


In [ ]:
full_combined_df['LDA_clean_text'] = full_combined_df['cleaned_tweet_text'].apply(lambda x : ' '.join(i for i in x.split() if  i not in tweet_search_keyword  ))


 Getting Date from the timestamp

In [ ]:
full_combined_df['created_at'] = full_combined_df['created_at'].astype('datetime64[ns]').dt.date
# full_combined_df['created_at'] = full_combined_df['created_at'].

In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,LDA_clean_text
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,vaccines covid seems turned,seems turned
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,taking beast vaccine drops,taking beast drops
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,ventilation supply outside accordance ashrae d...,ventilation supply outside accordance ashrae d...
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,breaking trump announced today overrule vaccin...,breaking trump announced today overrule disagr...
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,corona testing george floyds memorial sundays ...,testing george floyds memorial sundays 124pm b...


## EMPATH

https://github.com/Ejhfast/empath-client

In [ ]:

empath_mod = Empath()



Creating 10 Category with few added seed words and in response there are more associated by EMPATH neural embedding

The Categories and majority of seed words were taken from below Paper

Chopra, H., Vashishtha, A., Pal, R., Ashima, Tyagi, A., & Sethi, T. (2021). Mining Trends of COVID-19 Vaccine Beliefs on Twitter with Lexical Embeddings. ArXiv, abs/2104.01131.


In [ ]:
empath_mod.create_category("Misinformation",["propaganda", "conspiracy","fraud", "fake", "poison", "deception", 
                                  "plot", "trick", "plan", "frame-up", "false"])

["scheme", "method", "conspiracy", "theory", "trickery", "motive", "illusions", "purpose", "hoax", "deception", "killing", "false", "potential", "basis", "master_plan", "attachment", "mindset", "methods", "tactic", "proof", "delusion", "claims", "speculation", "schemes", "necessity", "villain", "part", "ruse", "complication", "mutation", "consequence", "act", "ploy", "downfall", "failure", "hypothesis", "catalyst", "fraud", "otherwise", "corrupt", "appeal", "capability", "phenomenon", "justice", "capabilities", "blackmail", "might", "ultimately", "theories", "aspects", "threat", "sources", "Furthermore", "illusion", "conversion", "vampirism", "significance", "time_travel", "creations", "accidental", "glitch", "probability", "human_nature", "free_will", "experiment", "representation", "context", "scam", "corruption", "potential", "origins", "victim", "extent", "limitations", "association", "origin", "essentially", "moral", "scenario", "conflict", "last_resort", "sorcery", "resolution", 

In [ ]:
empath_mod.create_category("Hesitation",["anxious", "nervous", "fear",
"consequences", "uncertain","reluctant",
"hesitation", "suspicion", "harm"])

["wary", "fearful", "suspicion", "afraid", "uncertain", "threat", "knowing", "fear", "anxious", "frightened", "reluctance", "unsure", "intentions", "uncertainty", "certainty", "fear", "reluctant", "hostility", "doubt", "cautious", "possibility", "worried", "danger", "hesitant", "doubts", "uneasy", "terrified", "apprehensive", "situation", "certain", "dilemma", "fearing", "reasoning", "Yet", "frightened", "notion", "emotion", "Caine", "caution", "Yet", "remorse", "curious", "doubtful", "rejection", "troubled", "consequences", "intent", "angry", "hopeful", "intention", "resistance", "relieved", "feared", "animosity", "anxiety", "guilt", "questioning", "unwilling", "curiosity", "unhappy", "guilty", "Syndil", "unease", "compelled", "uneasiness", "current_state", "simply", "question", "scared", "motives", "suspicious", "confrontation", "current_situation", "terrified", "wanting", "disappointed", "conviction", "predicament", "expectation", "pity", "surely", "determined", "weary", "Neverthele

In [ ]:
empath_mod.create_category("Sorrow", ["sad", "hopeless", "worst","dissatisfaction",
"disappointment", "setback"])

["sad", "hopeless", "hurt", "unhappy", "miserable", "terrible", "disappointment", "pity", "guilty", "sadness", "horrible", "angry", "guilt", "depressed", "worst", "betrayal", "upset", "heartbreaking", "heartbroken", "regret", "ashamed", "heartbreak", "denial", "disappointed", "rejection", "awful", "saddened", "truly", "sick", "loneliness", "heart_break", "sorrow", "scared", "failure", "Yet", "grief", "bad", "sympathy", "mixed_emotions", "disappointed", "knowing", "unhappiness", "happy", "emotional", "Yet", "shameful", "honestly", "devastating", "worse", "reason", "pitiful", "though", "hurt", "despair", "Pity", "humiliation", "heartache", "happiness", "bitter", "misery", "distraught", "terrified", "though", "pitied", "dreadful", "insecurity", "pathetic", "afraid", "emotion", "hurt", "heartbreaking", "fear", "suffering", "dissapointment", "worried", "hatred", "infatuation", "remorse", "so_much_pain", "depression", "unwanted", "depressed", "Though", "depressing", "shame", "dying", "anymor

In [ ]:
empath_mod.create_category("Faith", ["faith", "optimism", "vaccines","trust","belief",
"work", "assurance",
"grateful"])

["faith", "truly", "trust", "potential", "understand", "grateful", "accept", "free_will", "knowledge", "benefit", "trust", "Yet", "belief", "sacrifice", "loyalty", "human_life", "certainty", "independence", "commitment", "morals", "good_intentions", "redemption", "sacrifices", "downfall", "regardless", "succeed", "closure", "believed", "rely", "ambition", "potential", "responsibility", "mindset", "depend", "judgement", "willing", "fear", "Because", "surely", "Therefore", "compromise", "hardship", "risks", "strive", "extremes", "guidance", "faults", "virtue", "achieve", "good_fortune", "Though", "selfishly", "measure", "Regardless", "doubts", "limitations", "trusted", "responsibilities", "meant", "understood", "immortality", "resent", "capacity", "necessity", "benefit", "greater_good", "justice", "hope", "assure", "hardships", "common_sense", "saving", "failures", "Perhaps", "humanity", "simple_fact", "certainly", "consolation", "assurance", "experience", "believe", "devotion", "ignoran

In [ ]:
empath_mod.create_category("Contentment", ["satisfy", "glad", "proud",
"gratitude", "great", "joy"])


["happy", "proud", "grateful", "glad", "happiness", "thankful", "joy", "pride", "pleased", "good", "great", "thrilled", "gratitude", "truly", "wonderful", "appreciative", "blessed", "kindness", "delighted", "overjoyed", "achievement", "disappointed", "accomplishment", "hope", "knowing", "ecstatic", "much", "praise", "contented", "actually", "loved", "understanding", "honored", "generosity", "well", "greatful", "appreciate", "good", "enjoyment", "unhappy", "acceptance", "wish", "enthusiasm", "best", "pleased", "Yet", "Because", "satisfactory", "certainly", "success", "loving", "help", "satisfaction", "certainty", "accept", "satisfied", "positivity", "independence", "appreciated", "consolation", "enough", "wanted", "understood", "meant", "appreciation", "willing", "own", "much", "rejoice", "benefit", "ashamed", "generous", "delighted", "accepting", "overjoyed", "worthy", "worth", "though", "bravery", "compassion", "true_happiness", "special", "optimism", "amazing", "excited", "hard_work"

In [ ]:
empath_mod.create_category("Anticipation", ["anticipate", "urgently","expectancy",
"priority", "quick", "await"]) 

["crucial", "anticipate", "prepared", "vital", "await", "ensure", "quick", "determined", "receive", "thorough", "delay", "guaranteed", "calculated", "surely", "conclude", "Therefore", "required", "ensured", "aid", "allow", "priority", "dictated", "resolution", "determine", "imperative", "critical", "certain", "essential", "survival", "necessity", "quickest", "brief", "require", "objective", "resolved", "delaying", "necessary", "eager", "prepare", "conduct", "heed", "protocol", "depart", "requires", "insure", "compromised", "Perhaps", "yield", "execute", "prepared", "determining", "ensuring", "return", "prolong", "Given", "initiate", "reserve", "however", "advising", "sufficient", "eliminate", "hindrance", "instruction", "successful", "consult", "determines", "immediate", "order", "obtain", "concerning", "summons", "enable", "progress", "Zanoba", "campaign", "hopefully", "might", "limited", "accounted", "reinforce", "demand", "provide", "permit", "motivated", "able", "calculate", "final

In [ ]:
empath_mod.create_category("Rage", ["angry", "annoyance", "hate",
"mad", "pathetic"])

["angry", "mad", "upset", "annoyed", "pissed", "stupid", "annoyed", "hate", "frustrated", "furious", "upset", "scared", "sad", "pathetic", "disgusted", "jealous", "irritated", "pissed", "seriously", "hurt", "unhappy", "mean", "irritated", "guilty", "hating", "care", "annoying", "bad", "selfish", "disappointed", "hurt", "sick", "honestly", "angered", "horrible", "ashamed", "serious", "rude", "worried", "frustrated", "understand", "why", "aggravated", "Because", "upset", "infuriated", "insulted", "depressed", "annoying", "dissapointed", "embarrassed", "hated", "hurt", "stubborn", "ridiculous", "anymore", "jerk", "heartbroken", "blame", "disappointed", "mean", "happy", "defensive", "harsh", "awful", "disgusted", "terrible", "childish", "scared", "confused", "Honestly", "miserable", "wrong", "unfair", "obvious", "crazy", "reason", "insulting", "idiotic", "pissy", "knowing", "afraid", "dumb", "hates", "so_much_hate", "offended"]


In [ ]:
empath_mod.create_category("Vaccine Rollout", ["vaccinate", "distribution",
"supply", "mass", "dose", "vaccine rollout","vaccine campaign",
"vaccination drive"])

["supply", "amounts", "large_amount", "portion", "large_amounts", "mass", "variety", "masses", "various_types", "massive_amounts", "minerals", "amount", "stocks", "disposal", "trading", "abundance", "materials", "product", "weaponry", "metals", "plethora", "grain", "craft", "sorts", "housing", "maintenance", "production", "functions", "areas", "multitude", "majority", "primarily", "properties", "factories", "substances", "quantity", "massive_amount", "various", "certain_amount", "vessels", "resources", "providing", "provides", "stock", "facilities", "wide_range", "form", "requiring", "other_types", "goods", "cases", "reserve", "income", "most", "function", "livestock", "propaganda", "surplus", "fair_amount", "limited", "mechanics", "transport", "components", "profit", "bulk", "manipulation", "markets", "factory", "merchants", "shortage", "required", "portions", "technology", "constructing", "vessel", "bacteria", "bases", "consumption", "trade", "machinery", "addition", "other_type", "p

In [ ]:
empath_mod.create_category("Inequities", ["socioeconomic", "deprive",
"racial_injustice", "racism",
"underrepresented"])

["deprive", "racism", "discrimination", "glorify", "influencing", "constitute", "prejudice", "outweigh", "harbor", "condone", "equality", "detrimental", "homosexuality", "spiritual", "such_people", "exploitation", "Muslims", "derive", "morality", "corrupt", "human_nature", "abstinence", "merit", "benefited", "yagya", "neglect", "synonymous", "delusions", "influences", "immoral", "deserving", "democracy", "political", "condemn", "benefits", "vices", "morals", "unjust", "attribute", "moral", "conformity", "subject", "worst_kind", "sexism", "conform", "extremes", "degrade", "causes", "exclude", "terrorism", "satisfies", "most_cases", "prejudices", "thrives", "persecution", "shortcomings", "limitation", "upper_class", "degraded", "ridicule", "universal", "glorified", "pettiness", "influence", "contributing", "believers", "discriminate", "deserving", "worshipper", "associated", "yog", "conducive", "other_type", "corrupting", "incite", "successes", "transcend", "representation", "influences"

In [ ]:
empath_mod.create_category("Health Effects", ["nausea","swelling","muscle pain", "vomitting","chills","malaise","high temperature",
                "sneeze",   "cough", "nasal congestion", "dizziness","headache", "fatigue", "inflammation", "fever", "bodyache","drowsiness"])

["nausea", "migraine", "dizziness", "headache", "drowsiness", "tiredness", "numbness", "fatigue", "burning_sensation", "dull_ache", "burning_feeling", "ache", "excruciating_pain", "intense_pain", "fever", "exhaustion", "extreme_pain", "pain", "numb_feeling", "vertigo", "soreness", "throbbing", "horrible_pain", "stinging", "heaviness", "burning", "tremors", "anxiety", "burn", "immense_pain", "pains", "cold_feeling", "dull_pain", "coldness", "unbearable_pain", "sting", "familiar_feeling", "aches", "aching", "burning_pain", "terrible_pain", "vomiting", "swelling", "headaches", "massive_headache", "slight_pain", "Exhaustion", "ache", "throbbing_pain", "weariness", "slight_headache", "aching_pain", "dull_throb", "sleepiness", "blood_loss", "euphoria", "agonizing_pain", "stinging", "spasm", "dizzy", "sore_throat", "cramps", "uncomfortable_feeling", "stinging", "tremor", "numbed", "severe_pain", "entire_body", "woozy", "swelling", "stiffness", "shaking", "coughing", "numb", "strange_sensation

In [ ]:
categories = ['Hesitation', 'Sorrow', 
        'Faith', 'Contentment', 'Anticipation', 
        'Rage',   'Vaccine Rollout','Inequities',
           
         'Health Effects','Misinformation'
          ]

In [ ]:
len(categories)

10

In [ ]:
# categories = ['anticipation', 'positive_emotion', 
#         'contentment', 'negative_emotion', 'disappointment', 
#         'fear',   'trust','science',
#           'confusion', 
#          'health','optimism','misinformation',
#           'government']


In [ ]:
full_combined_df['EMPATH_scores'] = full_combined_df['cleaned_tweet_text'].apply(lambda x: empath_mod.analyze(x, categories= categories, normalize=True))


In [ ]:
# full_combined_df['EMPATH_text_scores'] = full_combined_df['text'].apply(lambda x: empath_mod.analyze(x, categories= categories, normalize=True))


In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,cleaned_tweet_text,LDA_clean_text,EMPATH_scores
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,vaccines covid seems turned,seems turned,"{'Hesitation': 0.0, 'Sorrow': 0.0, 'Faith': 0...."
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,taking beast vaccine drops,taking beast drops,"{'Hesitation': 0.0, 'Sorrow': 0.0, 'Faith': 0...."
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,ventilation supply outside accordance ashrae d...,ventilation supply outside accordance ashrae d...,"{'Hesitation': 0.0, 'Sorrow': 0.0, 'Faith': 0...."
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,breaking trump announced today overrule vaccin...,breaking trump announced today overrule disagr...,"{'Hesitation': 0.0, 'Sorrow': 0.0, 'Faith': 0...."
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,corona testing george floyds memorial sundays ...,testing george floyds memorial sundays 124pm b...,"{'Hesitation': 0.0, 'Sorrow': 0.0, 'Faith': 0...."


In [ ]:
def separate_columns(data,column_str,  column, key_val, final_col_name, column_keys):
  for category in column_keys:
    data[column_str + str(category)] = data[column].apply(lambda x: x[category] if category in x  else 0 )
  
  if final_col_name == 'LDA_topic':
    data[final_col_name] = data[column].apply(lambda x: max(zip(x.values(),x.keys()))[key_val] +1 )
  else:
    data[final_col_name] = data[column].apply(lambda x: max(zip(x.values(),x.keys()))[key_val])
  
  return data


In [ ]:
categories

['Hesitation',
 'Sorrow',
 'Faith',
 'Contentment',
 'Anticipation',
 'Rage',
 'Vaccine Rollout',
 'Inequities',
 'Health Effects',
 'Misinformation']

In [ ]:
full_combined_df = separate_columns(full_combined_df, '', 'EMPATH_scores', 1, 'final_category',categories)


In [ ]:

# full_combined_df = separate_columns(full_combined_df, 'original_', 'EMPATH_text_scores', 1, 'final_category_original',categories)


In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,...,Sorrow,Faith,Contentment,Anticipation,Rage,Vaccine Rollout,Inequities,Health Effects,Misinformation,final_category
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,Vaccine Rollout
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout


In [ ]:
full_combined_df['final_category'].value_counts()

Vaccine Rollout    2897
Contentment         299
Anticipation        273
Faith               214
Misinformation      213
Sorrow              206
Hesitation          113
Rage                105
Inequities           74
Health Effects       36
Name: final_category, dtype: int64

In [ ]:
# full_combined_df['final_category_original'].value_counts()


In [ ]:
full_combined_df.to_csv("All_data_with_EMPATH.csv")

In [ ]:
full_combined_df.columns

Index(['created_at', 'text', 'source', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count', 'country', 'full_name', 'Timeline',
       'cleaned_tweet_text', 'LDA_clean_text', 'EMPATH_scores', 'Hesitation',
       'Sorrow', 'Faith', 'Contentment', 'Anticipation', 'Rage',
       'Vaccine Rollout', 'Inequities', 'Health Effects', 'Misinformation',
       'final_category'],
      dtype='object')

## LDA

Lemmetizing For LDA

In [ ]:

wordnetLemma = nltk.WordNetLemmatizer()
full_combined_df['Lematized'] = full_combined_df['LDA_clean_text'].apply(lambda x :[wordnetLemma.lemmatize(word) for word in x.split()])


Creating the dictionary corpus from the Lematized text and filtering the dictionary from 10K words to 500 words

In [ ]:
 
corpus_dict = corpora.Dictionary(full_combined_df['Lematized'], prune_at= 5000)

len(corpus_dict)

9849

In [ ]:

corpus_dict.filter_extremes( no_above=0.9, keep_n=500)
len(corpus_dict)

500

In [ ]:
for k,v in corpus_dict.items():
 
    print(k,v)

0 seems
1 taking
2 building
3 first
4 outside
5 sarscov2
6 supply
7 transmission
8 announced
9 breaking
10 company
11 today
12 trump
13 please
14 testing
15 cell
16 effect
17 infection
18 possible
19 protein
20 viral
21 working
22 colleague
23 healthcare
24 partner
25 think
26 worker
27 highly
28 administration
29 covid19
30 created
31 speed
32 therapeutic
33 virus
34 death
35 january
36 nearly
37 within
38 country
39 every
40 offering
41 president
42 week
43 would
44 biden
45 fight
46 health
47 house
48 moment
49 nation
50 weekend
51 white
52 american
53 believe
54 normal
55 thing
56 election
57 debate
58 life
59 measure
60 million
61 others
62 people
63 political
64 protect
65 thats
66 dying
67 member
68 night
69 number
70 service
71 still
72 time
73 anyone
74 medium
75 question
76 safety
77 round
78 school
79 second
80 reason
81 there
82 business
83 discus
84 impact
85 including
86 response
87 hearing
88 fauci
89 reading
90 work
91 change
92 mask
93 across
94 grateful
95 science
96 

Using the dictionary corpus we create a Document Term Matrix

In [ ]:

dtm_matrix = [corpus_dict.doc2bow(doc) for doc in full_combined_df['Lematized']]


In [ ]:
full_combined_df['dtm_matrix'] = full_combined_df['Lematized'].apply(lambda x : corpus_dict.doc2bow(x))

In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,...,Contentment,Anticipation,Rage,Vaccine Rollout,Inequities,Health Effects,Misinformation,final_category,Lematized,dtm_matrix
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[seems, turned]","[(0, 1)]"
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[taking, beast, drop]","[(1, 1)]"
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.0,0.05,0.0,0.0,0.0,Vaccine Rollout,"[ventilation, supply, outside, accordance, ash...","[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]"
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[breaking, trump, announced, today, overrule, ...","[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]"
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[testing, george, floyds, memorial, sunday, 12...","[(13, 1), (14, 1)]"


Running and training The LDA model 

In [ ]:

lda = LdaMulticore(dtm_matrix, num_topics=5, id2word = corpus_dict, passes=100, chunksize= 500)

In [ ]:
for k, v in lda.print_topics(num_topics=5):
  print(" Topic ", k)
  print(v)

 Topic  0
0.088*"covid19" + 0.038*"first" + 0.037*"fully" + 0.035*"community" + 0.030*"school" + 0.024*"biden" + 0.020*"health" + 0.020*"public" + 0.019*"center" + 0.018*"president"
 Topic  1
0.140*"sarscov2" + 0.074*"covid19" + 0.048*"virus" + 0.028*"infection" + 0.019*"variant" + 0.018*"spread" + 0.018*"response" + 0.018*"patient" + 0.016*"effective" + 0.014*"antibody"
 Topic  2
0.145*"covid19" + 0.056*"today" + 0.027*"first" + 0.025*"health" + 0.021*"appointment" + 0.019*"county" + 0.018*"getting" + 0.017*"clinic" + 0.016*"worker" + 0.011*"eligible"
 Topic  3
0.080*"covid19" + 0.074*"people" + 0.049*"state" + 0.025*"available" + 0.024*"million" + 0.021*"dos" + 0.019*"everyone" + 0.018*"country" + 0.018*"science" + 0.016*"family"
 Topic  4
0.047*"effect" + 0.039*"people" + 0.037*"trump" + 0.026*"still" + 0.025*"going" + 0.022*"youre" + 0.022*"right" + 0.021*"would" + 0.021*"think" + 0.020*"thing"


In [ ]:
full_combined_df['LDA_topic_vals'] = full_combined_df['dtm_matrix'].apply(lambda x : dict(lda[x]))

In [ ]:

full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,...,Anticipation,Rage,Vaccine Rollout,Inequities,Health Effects,Misinformation,final_category,Lematized,dtm_matrix,LDA_topic_vals
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[seems, turned]","[(0, 1)]","{0: 0.1000013, 1: 0.10127927, 2: 0.10000103, 3..."
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[taking, beast, drop]","[(1, 1)]","{0: 0.59856844, 1: 0.10143015, 2: 0.10000039, ..."
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,...,0.0,0.0,0.05,0.0,0.0,0.0,Vaccine Rollout,"[ventilation, supply, outside, accordance, ash...","[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]","{0: 0.028977264, 1: 0.3144106, 2: 0.45649832, ..."
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[breaking, trump, announced, today, overrule, ...","[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]","{0: 0.03388661, 1: 0.033333655, 2: 0.67067724,..."
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,...,0.0,0.0,0.00,0.0,0.0,0.0,Vaccine Rollout,"[testing, george, floyds, memorial, sunday, 12...","[(13, 1), (14, 1)]","{0: 0.39999315, 1: 0.06666712, 2: 0.066667005,..."


Converting the LDA model weights of Documents into columns

In [ ]:
full_combined_df =  separate_columns(full_combined_df, 'Topic_', 'LDA_topic_vals', 1, 'LDA_topic', full_combined_df['LDA_topic_vals'].iloc[0].keys())

In [ ]:
full_combined_df.head()

,created_at,text,source,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,country,full_name,Timeline,...,final_category,Lematized,dtm_matrix,LDA_topic_vals,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,LDA_topic
0,2020-09-06,I’m all for vaccines but this COVID one seems ...,Twitter for iPhone,8344,60,27251,352,United States,"Atlanta, GA",Before_Sep,...,Vaccine Rollout,"[seems, turned]","[(0, 1)]","{0: 0.1000013, 1: 0.10127927, 2: 0.10000103, 3...",0.100001,0.101279,0.100001,0.100001,0.598717,5
1,2020-09-28,Y’all taking that mark of the beast vaccine wh...,Twitter for iPhone,122,22,677,14,United States,"Atlanta, GA",Before_Sep,...,Vaccine Rollout,"[taking, beast, drop]","[(1, 1)]","{0: 0.59856844, 1: 0.10143015, 2: 0.10000039, ...",0.598568,0.101430,0.100000,0.100000,0.100001,1
2,2020-09-17,2. Ventilation: \nA good supply of outside air...,Twitter for Mac,0,1,0,0,United States,"Atlanta, GA",Before_Sep,...,Vaccine Rollout,"[ventilation, supply, outside, accordance, ash...","[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]","{0: 0.028977264, 1: 0.3144106, 2: 0.45649832, ...",0.028977,0.314411,0.456498,0.028809,0.171305,3
3,2020-09-24,BREAKING: Trump announced today he will overru...,Twitter for iPhone,7185,1285,23549,1049,United States,"Minneapolis, MN",Before_Sep,...,Vaccine Rollout,"[breaking, trump, announced, today, overrule, ...","[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]","{0: 0.03388661, 1: 0.033333655, 2: 0.67067724,...",0.033887,0.033334,0.670677,0.033561,0.228541,3
4,2020-09-13,They doing free Corona testing at George Floyd...,Twitter for Android,509,13,943,1,United States,"Minneapolis, MN",Before_Sep,...,Vaccine Rollout,"[testing, george, floyds, memorial, sunday, 12...","[(13, 1), (14, 1)]","{0: 0.39999315, 1: 0.06666712, 2: 0.066667005,...",0.399993,0.066667,0.066667,0.067427,0.399246,1


In [ ]:
full_combined_df['LDA_topic'].value_counts()

3    1082
5     955
2     902
4     763
1     728
Name: LDA_topic, dtype: int64

Using PyLDAvis library to create a vizualization and anlyze if any of the topics are overlapping each other 

In [ ]:

pyLDAvis.enable_notebook()

LDA_vizualization = gensimvis.prepare(lda, dtm_matrix, corpus_dict)
pyLDAvis.save_html(LDA_vizualization, './LDA_vizualization_'+ str(5) +'.html')
LDA_vizualization

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.166695 -0.136735       1        1  23.301124
3     -0.017121  0.079845       2        1  19.831832
1      0.029935  0.311288       3        1  19.685635
4     -0.372823 -0.113082       4        1  18.988731
0      0.193313 -0.141316       5        1  18.192678, topic_info=         Term         Freq        Total Category  logprob  loglift
5    sarscov2   669.000000   669.000000  Default  30.0000  30.0000
62     people   541.000000   541.000000  Default  29.0000  29.0000
29    covid19  1946.000000  1946.000000  Default  28.0000  28.0000
11      today   347.000000   347.000000  Default  27.0000  27.0000
265     state   234.000000   234.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
29    covid19   388.463375  1946.416775   Topic5  -2.4315   0.0926
46     health    89.285390   231.796067   Topic5  -3.9019   0.7501
314     thank    46.517501    93.328698   Topic5  -4.5539   1.0079
42       week    35.486324    56.817659   Topic5  -4.8246   1.2335
1      taking    34.663660    50.127857   Topic5  -4.8480   1.3353

[229 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
355       1  0.971026      administer
28        1  0.972782  administration
398       4  0.661976         already
398       5  0.330988         already
469       5  0.968586         amazing
...     ...       ...             ...
43        3  0.356034           would
43        4  0.640861           would
128       3  0.228956            year
128       4  0.763187            year
221       4  0.991382           youre

[236 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 5, 1])

Output with LDA


In [ ]:
full_combined_df.to_csv("All_data_with_LDA.csv")

SVM


file:///Users/himanideshpande/Downloads/ijerph-18-05993.pdf

Using TFID for vectorizing the text which will help in getting better performance for SVM

In [ ]:

vectorizer_mod = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
vectors = vectorizer_mod.fit_transform(full_combined_df['cleaned_tweet_text'])


In [ ]:
Y = full_combined_df['final_category']

Splitting the train and test set in 80-20 proportion

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(vectors, Y, test_size=0.2, random_state=0)

Training the SVM model

In [ ]:

svc = SVC(C=10, gamma=0.1)

svc.fit(X_train, y_train)

predictions = svc.predict(X_test)


In [ ]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, predictions) * 100))

scores = precision_recall_fscore_support(y_test, predictions, warn_for=('precision', 'recall', 'f-score'),average='weighted')

print("\nPrecision: {:.2f}".format(scores[0]* 100))
print("\nRecall: {:.2f}".format(scores[1]* 100))
print("\nF1 Score: {:.2f}".format(scores[2]* 100))

Accuracy: 80.93%

Precision: 81.08

Recall: 80.93

F1 Score: 78.84


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Using GridSearch for HyperTuning

In [ ]:

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
 
grid = GridSearchCV(SVC(), param_grid, refit = True,verbose =0)
 
grid.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear']})

Getting the Best Parameter and Estimator

In [ ]:
print(grid.best_params_)
 

print(grid.best_estimator_)

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=1000, gamma=0.001)


Creating the new Model after GridSearch Tuning

In [ ]:
svc = grid.best_estimator_

svc.fit(X_train, y_train)

predictions = svc.predict(X_test)



In [ ]:
from sklearn.metrics import recall_score

In [ ]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, predictions) * 100))
scores = precision_recall_fscore_support(y_test, predictions, warn_for=('precision', 'recall', 'f-score'),average='weighted')

print("\nPrecision: {:.2f}".format(scores[0]* 100))
print("\nRecall: {:.2f}".format(scores[1]* 100))
print("\nF1 Score: {:.2f}".format(scores[2]* 100))

Accuracy: 81.15%

Precision: 81.03

Recall: 81.15

F1 Score: 79.26


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## RandomForest

In [ ]:

random_forrest_mod = RandomForestClassifier(n_estimators=200,random_state=0)
random_forrest_mod.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [ ]:
predictions = random_forrest_mod.predict(X_test)

In [ ]:
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, predictions) * 100))
scores = precision_recall_fscore_support(y_test, predictions, warn_for=('precision', 'recall', 'f-score'),average='weighted')

print("\nPrecision: {:.2f}".format(scores[0]* 100))
print("\nRecall: {:.2f}".format(scores[1]* 100))
print("\nF1 Score: {:.2f}".format(scores[2]* 100))

Accuracy: 85.78%

Precision: 85.72

Recall: 85.78

F1 Score: 85.01


GridSearch Hypertuning for RandomForrest

In [ ]:


param_grid = {
    'random_state': [0,42],
    'max_depth': [None, 80, 90, 100, 110],
    'n_estimators': [100, 200, 300, 1000]
}

grid_RandomForrest = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          refit = True, verbose = 0)

 
grid_RandomForrest.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 80, 90, 100, 110],
                         'n_estimators': [100, 200, 300, 1000],
                         'random_state': [0, 42]})

In [ ]:
print(grid_RandomForrest.best_params_)
 

print(grid_RandomForrest.best_estimator_)

{'max_depth': None, 'n_estimators': 300, 'random_state': 0}
RandomForestClassifier(n_estimators=300, random_state=0)


Creating the new RandomForrest Model after Grid Search

In [ ]:

random_forrest_mod = grid_RandomForrest.best_estimator_
random_forrest_mod.fit(X_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=0)

In [ ]:
predictions = random_forrest_mod.predict(X_test)
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, predictions) * 100))
scores = precision_recall_fscore_support(y_test, predictions, warn_for=('precision', 'recall', 'f-score'),average='weighted')

print("\nPrecision: {:.2f}".format(scores[0]* 100))
print("\nRecall: {:.2f}".format(scores[1]* 100))
print("\nF1 Score: {:.2f}".format(scores[2]* 100))


Accuracy: 85.67%

Precision: 85.60

Recall: 85.67

F1 Score: 84.88


Saving Output

In [ ]:
full_combined_df.to_csv("Tweet_analysis_Output.csv")

References:

Tweepy:

https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all

https://docs.tweepy.org/en/stable/client.html#tweets

https://developer.twitter.com/en/docs/twitter-api/tweets/counts/integrate/build-a-query

DATA Processing:

https://stackoverflow.com/questions/50830214/remove-usernames-from-twitter-data-using-python


https://www.nltk.org/howto/wordnet.html

EMPATH:

https://github.com/Ejhfast/empath-client

LDA:


https://radimrehurek.com/gensim/models/ldamulticore.html

https://github.com/RaRe-Technologies/gensim

https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html

https://pyldavis.readthedocs.io/en/latest/readme.html

SVM:

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html

RandomForrest:

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html